In [14]:
import numpy as np
import pandas as pd
import sklearn
import spacy
import re
from nltk.corpus import gutenberg
import nltk
import warnings
warnings.filterwarnings("ignore")
from nltk import word_tokenize

nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/abilenky/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [10]:
sentences = ["The Lumberjack Song is the funniest Monty Python bit; I can't think of it without laughing.",
"I would rather put strawberries on my ice cream for dessert; they have the best taste.",
"The taste of caramel is a fantastic accompaniment to tasty mint ice cream."]

In [11]:
sentences

["The Lumberjack Song is the funniest Monty Python bit; I can't think of it without laughing.",
 'I would rather put strawberries on my ice cream for dessert; they have the best taste.',
 'The taste of caramel is a fantastic accompaniment to tasty mint ice cream.']

In [33]:
# Get rid of stop words and punctuation,
# and lemmatize the tokens
tokens = []
for sentence in sentences:
    doc = nlp(sentence)
    tokens.append([token.lemma_.lower() for token in doc if not token.is_punct and not token.is_stop])
    
print(tokens)

[['lumberjack', 'song', 'funny', 'monty', 'python', 'bit', 'think', 'laugh'], ['strawberry', 'ice', 'cream', 'dessert', 'good', 'taste'], ['taste', 'caramel', 'fantastic', 'accompaniment', 'tasty', 'mint', 'ice', 'cream']]


In [40]:
cleaned = [' '.join(sent) for sent in tokens]
cleaned_df = pd.DataFrame(cleaned)
cleaned_df.columns = ['text']
cleaned_df

,text
0,lumberjack song funny monty python bit think l...
1,strawberry ice cream dessert good taste
2,taste caramel fantastic accompaniment tasty mi...


### 1. Generate the TF-IDF vectors for the last three sentences of the example from the beginning of this checkpoint 

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, norm=u'l2', smooth_idf=True)

vectors = vectorizer.fit_transform(cleaned_df.text)
pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())

,accompaniment,bit,caramel,cream,dessert,fantastic,funny,good,ice,laugh,lumberjack,mint,monty,python,song,strawberry,taste,tasty,think
0,0.000000,0.353553,0.000000,0.000000,0.000000,0.000000,0.353553,0.000000,0.000000,0.353553,0.353553,0.000000,0.353553,0.353553,0.353553,0.000000,0.000000,0.000000,0.353553
1,0.000000,0.000000,0.000000,0.349498,0.459548,0.000000,0.000000,0.459548,0.349498,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.459548,0.349498,0.000000,0.000000
2,0.385323,0.000000,0.385323,0.293048,0.000000,0.385323,0.000000,0.000000,0.293048,0.000000,0.000000,0.385323,0.000000,0.000000,0.000000,0.000000,0.293048,0.385323,0.000000


### 2. In the 2-grams example above, you only used 2-grams as your features. This time, use both 1-grams and 2-grams together as your feature set. Run the same models as in the example and compare the results.

In [44]:
# Utility function for standard text cleaning
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation that spaCy doesn't
    # recognize: the double dash --. Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = ' '.join(text.split())
    return text

In [45]:
# Load and clean the data
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [46]:
# Parse the cleaned novels. This can take some time.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [47]:
# Group into sentences
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one DataFrame
sentences = pd.DataFrame(alice_sents + persuasion_sents, columns = ["text", "author"])
sentences.head()

,text,author
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


In [48]:
# Get rid of stop words and punctuation,
# and lemmatize the tokens
for i, sentence in enumerate(sentences["text"]):
    sentences.loc[i, "text"] = " ".join(
        [token.lemma_ for token in sentence if not token.is_punct and not token.is_stop])

In [49]:
vectorizer = TfidfVectorizer(
    max_df=0.5, min_df=2, use_idf=True, norm=u'l2', smooth_idf=True, ngram_range=(1,2))


# Applying the vectorizer
X = vectorizer.fit_transform(sentences["text"])

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([tfidf_df, sentences[["text", "author"]]], axis=1)

# Keep in mind that log base 2 of 1 is 0,
# so a TF-IDF score of 0 indicates that the word was present once in that sentence.
sentences.head()

,abide,ability,able,able bear,able persuade,abominate,abroad,absence,absence home,absent,...,young people,young person,young sister,young woman,youth,youth say,zeal,zealous,text,author
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alice begin tired sit sister bank have twice p...,Carroll
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,consider mind hot day feel sleepy stupid pleas...,Carroll
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,remarkable Alice think way hear Rabbit,Carroll
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,oh dear,Carroll
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,oh dear,Carroll


In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

Y = sentences['author']
X = np.array(sentences.drop(['text','author'], 1))

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=123)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 0.912696063924238

Test set score: 0.8717265867731913
----------------------Random Forest Scores----------------------
Training set score: 0.9786919206865937

Test set score: 0.8708388814913449
----------------------Gradient Boosting Scores----------------------
Training set score: 0.8490677715300384

Test set score: 0.8326675543719485
